In [2]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappush, heappop
from copy import deepcopy
import time

# Solving the n**2 - 1 puzzle
I implemented the A* algorithm

In [3]:
# Helper functions
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_goal(state):
    """Check if the current state is the goal state."""
    return np.array_equal(state, goal_state)

## Which heuristic?
I tested various approaches:
- Manhattan distance
- The number of tiles in the _wrong_ place
- Manhattan distance + 2 * linear conflict 
- Manhattan distance - the number of tiles in the _right_ place
- Manhattan distance - 0.5 * the number of tiles in the _right_ place
- Manhattan distance - 2 * the number of tiles in the _right_ place
- Manhattan distance + 2 * linear conflict - the number of tiles in the _right_ place

I'll only implement _Manhattan distance - the number of tiles in the right place_ because from my tests it was clearly the best approach without any doubts (less time, less moves, and less explored states). I was surprised by this because the first three approaches I listed are the "norm" in approaching this problem, and I didn't expect my hybrid approach which came from a simple idea (I thought: Manhattan distance is not enough, I want to reward in some way the tiles in the right place) to be better. The last 3 approaches, I tried to see if I could improve even more the heuristic, but they were worse not better.

In [4]:
#Heuristics
def manhattan_distance(state):
    """Calculate the Manhattan distance heuristic."""
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Ignore the empty tile
                target_x, target_y = divmod(value - 1, PUZZLE_DIM)
                distance += abs(x - target_x) + abs(y - target_y)
    return distance

def linear_conflict(state):
    """Calculate linear conflict heuristic."""
    conflict = 0
    for i in range(PUZZLE_DIM):
        # Check row conflicts
        row = state[i, :]
        goal_row = [(val - 1) // PUZZLE_DIM for val in row if val != 0]
        conflict += sum(goal_row[j] == i and any(goal_row[k] < goal_row[j] for k in range(j)) for j in range(len(goal_row)))

        # Check column conflicts
        col = state[:, i]
        goal_col = [(val - 1) % PUZZLE_DIM for val in col if val != 0]
        conflict += sum(goal_col[j] == i and any(goal_col[k] < goal_col[j] for k in range(j)) for j in range(len(goal_col)))

    return conflict

def enhanced_heuristic(state):
    """Combine Manhattan distance with linear conflict."""
    return manhattan_distance(state) + 2 * linear_conflict(state)

def tiles_in_place(state):
    """Count tiles in their correct position."""
    correct = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            if state[x, y] != 0 and state[x, y] == goal_state[x, y]:
                correct += 1
    return correct
    #return PUZZLE_DIM**2 - correct

## A* function

In [5]:
# A* Search Algorithm
def solve_puzzle(initial_state):
    """Solve the n^2-1 puzzle using A* search."""
    # Priority queue
    open_list = []
    heappush(open_list, (0, initial_state.tobytes(), [], 0, None))  # Store states as bytes
    visited = set()
    total_states_evaluated = 0  # Cost metric

    while open_list:
        f, current_state_bytes, path, g, last_move = heappop(open_list)
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))

        total_states_evaluated += 1  # Increment cost

        # Check if goal
        if is_goal(current_state):
            # Convert states back to readable format
            readable_path = [np.frombuffer(state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM)) for state_bytes in path]
            return readable_path, total_states_evaluated

        visited.add(current_state_bytes)

        for move in available_actions(current_state):
            if last_move and move[1] == last_move[0]:  # Avoid reversing last move
                continue
            next_state = do_action(current_state, move)
            next_state_bytes = next_state.tobytes()
            if next_state_bytes not in visited:
                next_g = g + 1
                h = (manhattan_distance(next_state) - tiles_in_place(next_state))
                #h = enhanced_heuristic(next_state)
                # Append the serialized state to the path
                heappush(open_list, (next_g + h, next_state_bytes, path + [next_state_bytes], next_g, move))

    return None, total_states_evaluated  # No solution

## 3x3 

In [24]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in range(RANDOMIZE_STEPS):
    state = do_action(state, choice(available_actions(state)))
state

array([[2, 4, 5],
       [6, 0, 1],
       [8, 7, 3]])

In [25]:
# Solve and output
start_time = time.time()
solution, cost = solve_puzzle(state)
end_time = time.time()
delta = end_time - start_time
print("Solution Steps:")
x = 1
for step in solution:
    print(f"Step {x}:")
    x += 1
    print(step)

Solution Steps:
Step 1:
[[2 4 5]
 [0 6 1]
 [8 7 3]]
Step 2:
[[2 4 5]
 [8 6 1]
 [0 7 3]]
Step 3:
[[2 4 5]
 [8 6 1]
 [7 0 3]]
Step 4:
[[2 4 5]
 [8 0 1]
 [7 6 3]]
Step 5:
[[2 4 5]
 [0 8 1]
 [7 6 3]]
Step 6:
[[0 4 5]
 [2 8 1]
 [7 6 3]]
Step 7:
[[4 0 5]
 [2 8 1]
 [7 6 3]]
Step 8:
[[4 5 0]
 [2 8 1]
 [7 6 3]]
Step 9:
[[4 5 1]
 [2 8 0]
 [7 6 3]]
Step 10:
[[4 5 1]
 [2 8 3]
 [7 6 0]]
Step 11:
[[4 5 1]
 [2 8 3]
 [7 0 6]]
Step 12:
[[4 5 1]
 [2 0 3]
 [7 8 6]]
Step 13:
[[4 0 1]
 [2 5 3]
 [7 8 6]]
Step 14:
[[4 1 0]
 [2 5 3]
 [7 8 6]]
Step 15:
[[4 1 3]
 [2 5 0]
 [7 8 6]]
Step 16:
[[4 1 3]
 [2 0 5]
 [7 8 6]]
Step 17:
[[4 1 3]
 [0 2 5]
 [7 8 6]]
Step 18:
[[0 1 3]
 [4 2 5]
 [7 8 6]]
Step 19:
[[1 0 3]
 [4 2 5]
 [7 8 6]]
Step 20:
[[1 2 3]
 [4 0 5]
 [7 8 6]]
Step 21:
[[1 2 3]
 [4 5 0]
 [7 8 6]]
Step 22:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


In [26]:
print(f"It took {delta} seconds to run the code")
print(f"Quality (Number of Moves): {len(solution)}")
print(f"Cost (Total States Evaluated): {cost}")

It took 0.012010335922241211 seconds to run the code
Quality (Number of Moves): 22
Cost (Total States Evaluated): 406


## 4x4

In [27]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in range(RANDOMIZE_STEPS):
    state = do_action(state, choice(available_actions(state)))
state

array([[12, 11, 15, 14],
       [ 1,  3, 10,  0],
       [ 8, 13,  7,  6],
       [ 5,  9,  2,  4]])

In [28]:
# Solve and output
start_time = time.time()
solution, cost = solve_puzzle(state)
end_time = time.time()
delta = end_time - start_time
print("Solution Steps:")
x = 1
for step in solution:
    print(f"Step {x}:")
    x += 1
    print(step)

Solution Steps:
Step 1:
[[12 11 15 14]
 [ 1  3  0 10]
 [ 8 13  7  6]
 [ 5  9  2  4]]
Step 2:
[[12 11  0 14]
 [ 1  3 15 10]
 [ 8 13  7  6]
 [ 5  9  2  4]]
Step 3:
[[12  0 11 14]
 [ 1  3 15 10]
 [ 8 13  7  6]
 [ 5  9  2  4]]
Step 4:
[[ 0 12 11 14]
 [ 1  3 15 10]
 [ 8 13  7  6]
 [ 5  9  2  4]]
Step 5:
[[ 1 12 11 14]
 [ 0  3 15 10]
 [ 8 13  7  6]
 [ 5  9  2  4]]
Step 6:
[[ 1 12 11 14]
 [ 8  3 15 10]
 [ 0 13  7  6]
 [ 5  9  2  4]]
Step 7:
[[ 1 12 11 14]
 [ 8  3 15 10]
 [ 5 13  7  6]
 [ 0  9  2  4]]
Step 8:
[[ 1 12 11 14]
 [ 8  3 15 10]
 [ 5 13  7  6]
 [ 9  0  2  4]]
Step 9:
[[ 1 12 11 14]
 [ 8  3 15 10]
 [ 5  0  7  6]
 [ 9 13  2  4]]
Step 10:
[[ 1 12 11 14]
 [ 8  3 15 10]
 [ 5  7  0  6]
 [ 9 13  2  4]]
Step 11:
[[ 1 12 11 14]
 [ 8  3  0 10]
 [ 5  7 15  6]
 [ 9 13  2  4]]
Step 12:
[[ 1 12 11 14]
 [ 8  3 10  0]
 [ 5  7 15  6]
 [ 9 13  2  4]]
Step 13:
[[ 1 12 11  0]
 [ 8  3 10 14]
 [ 5  7 15  6]
 [ 9 13  2  4]]
Step 14:
[[ 1 12  0 11]
 [ 8  3 10 14]
 [ 5  7 15  6]
 [ 9 13  2  4]]
Step 15:
[[ 1

In [29]:
print(f"It took {delta} seconds to run the code")
print(f"Quality (Number of Moves): {len(solution)}")
print(f"Cost (Total States Evaluated): {cost}")

It took 141.194993019104 seconds to run the code
Quality (Number of Moves): 60
Cost (Total States Evaluated): 2849348


## Improving A* with weights
I easily solved the 3x3 and 4x4 problems, then I got stuck. Using weights makes it possible to solve more difficult problems: here you can see what happens to 4x4 when changing the weight. The quality goes slighly down (e.g. the number of move go up), but both the time and the number of explored states go drastically down.


In [6]:
# A* Search Algorithm 
def solve_puzzle_weight(initial_state, weight):
    """Solve the n^2-1 puzzle using A* search."""
    # Priority queue
    open_list = []
    heappush(open_list, (0, initial_state.tobytes(), [], 0, None))  # Store states as bytes
    visited = set()
    total_states_evaluated = 0  # Cost metric

    while open_list:
        f, current_state_bytes, path, g, last_move = heappop(open_list)
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))

        total_states_evaluated += 1  # Increment cost

        # Check if goal
        if is_goal(current_state):
            # Convert states back to readable format
            readable_path = [np.frombuffer(state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM)) for state_bytes in path]
            return readable_path, total_states_evaluated

        visited.add(current_state_bytes)

        for move in available_actions(current_state):
            if last_move and move[1] == last_move[0]:  # Avoid reversing last move
                continue
            next_state = do_action(current_state, move)
            next_state_bytes = next_state.tobytes()
            if next_state_bytes not in visited:
                next_g = g + 1
                h = weight * (manhattan_distance(next_state) - tiles_in_place(next_state))
                #h = enhanced_heuristic(next_state)
                # Append the serialized state to the path
                heappush(open_list, (next_g + h, next_state_bytes, path + [next_state_bytes], next_g, move))

    return None, total_states_evaluated  # No solution

In [31]:
# List of weight values to test
weights = [1, 1.5, 2, 3]

print(f"{'Weight':<10}{'Time (seconds)':<20}{'Quality (Moves)':<20}{'Cost (States Evaluated)'}")
print("="*70)  # Divider line for the table header

for weight in weights:
    start_time = time.time()
    solution, cost = solve_puzzle_weight(state, weight)
    end_time = time.time()

    delta = end_time - start_time
    quality = len(solution)

    print(f"{weight:<10}{delta:<20.4f}{quality:<20}{cost}")

Weight    Time (seconds)      Quality (Moves)     Cost (States Evaluated)
1         146.2022            60                  2849348
1.5       1.2023              68                  24476
2         0.8829              78                  19047
3         0.1413              94                  3031


## 5x5 (weighted A*)

In [32]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in range(RANDOMIZE_STEPS):
    state = do_action(state, choice(available_actions(state)))
state

array([[ 0, 11, 16, 17, 24],
       [20, 12, 14, 22, 19],
       [ 6, 10, 18,  1,  7],
       [15,  8,  5,  4, 21],
       [13, 23,  9,  2,  3]])

In [33]:
# Solve and output
weight = 3
start_time = time.time()
solution, cost = solve_puzzle_weight(state, weight)
end_time = time.time()
delta = end_time - start_time
print("Solution Steps:")
x = 1
for step in solution:
    print(f"Step {x}:")
    x += 1
    print(step)

Solution Steps:
Step 1:
[[11  0 16 17 24]
 [20 12 14 22 19]
 [ 6 10 18  1  7]
 [15  8  5  4 21]
 [13 23  9  2  3]]
Step 2:
[[11 16  0 17 24]
 [20 12 14 22 19]
 [ 6 10 18  1  7]
 [15  8  5  4 21]
 [13 23  9  2  3]]
Step 3:
[[11 16 17  0 24]
 [20 12 14 22 19]
 [ 6 10 18  1  7]
 [15  8  5  4 21]
 [13 23  9  2  3]]
Step 4:
[[11 16 17 22 24]
 [20 12 14  0 19]
 [ 6 10 18  1  7]
 [15  8  5  4 21]
 [13 23  9  2  3]]
Step 5:
[[11 16 17 22 24]
 [20 12 14  1 19]
 [ 6 10 18  0  7]
 [15  8  5  4 21]
 [13 23  9  2  3]]
Step 6:
[[11 16 17 22 24]
 [20 12 14  1 19]
 [ 6 10 18  4  7]
 [15  8  5  0 21]
 [13 23  9  2  3]]
Step 7:
[[11 16 17 22 24]
 [20 12 14  1 19]
 [ 6 10 18  4  7]
 [15  8  0  5 21]
 [13 23  9  2  3]]
Step 8:
[[11 16 17 22 24]
 [20 12 14  1 19]
 [ 6 10  0  4  7]
 [15  8 18  5 21]
 [13 23  9  2  3]]
Step 9:
[[11 16 17 22 24]
 [20 12  0  1 19]
 [ 6 10 14  4  7]
 [15  8 18  5 21]
 [13 23  9  2  3]]
Step 10:
[[11 16  0 22 24]
 [20 12 17  1 19]
 [ 6 10 14  4  7]
 [15  8 18  5 21]
 [13 23  9  

In [34]:
print(f"It took {delta} seconds to run the code")
print(f"Quality (Number of Moves): {len(solution)}")
print(f"Cost (Total States Evaluated): {cost}")

It took 11.965858221054077 seconds to run the code
Quality (Number of Moves): 246
Cost (Total States Evaluated): 181529


## 6x6 (weighted A*)

In [35]:
PUZZLE_DIM = 6
action = namedtuple('Action', ['pos1', 'pos2'])
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in range(RANDOMIZE_STEPS):
    state = do_action(state, choice(available_actions(state)))
state

array([[22, 34,  0, 28,  4, 11],
       [ 2, 32, 13,  3, 10,  1],
       [30, 14, 24, 19, 21,  6],
       [17, 27, 33, 31, 15,  9],
       [18, 35,  5,  8, 26, 20],
       [16, 12, 25,  7, 29, 23]])

In [36]:
# Solve and output
weight = 10
start_time = time.time()
solution, cost = solve_puzzle_weight(state, weight)
end_time = time.time()
delta = end_time - start_time
print("Solution Steps:")
x = 1
for step in solution:
    print(f"Step {x}:")
    x += 1
    print(step)

Solution Steps:
Step 1:
[[22  0 34 28  4 11]
 [ 2 32 13  3 10  1]
 [30 14 24 19 21  6]
 [17 27 33 31 15  9]
 [18 35  5  8 26 20]
 [16 12 25  7 29 23]]
Step 2:
[[ 0 22 34 28  4 11]
 [ 2 32 13  3 10  1]
 [30 14 24 19 21  6]
 [17 27 33 31 15  9]
 [18 35  5  8 26 20]
 [16 12 25  7 29 23]]
Step 3:
[[ 2 22 34 28  4 11]
 [ 0 32 13  3 10  1]
 [30 14 24 19 21  6]
 [17 27 33 31 15  9]
 [18 35  5  8 26 20]
 [16 12 25  7 29 23]]
Step 4:
[[ 2 22 34 28  4 11]
 [30 32 13  3 10  1]
 [ 0 14 24 19 21  6]
 [17 27 33 31 15  9]
 [18 35  5  8 26 20]
 [16 12 25  7 29 23]]
Step 5:
[[ 2 22 34 28  4 11]
 [30 32 13  3 10  1]
 [17 14 24 19 21  6]
 [ 0 27 33 31 15  9]
 [18 35  5  8 26 20]
 [16 12 25  7 29 23]]
Step 6:
[[ 2 22 34 28  4 11]
 [30 32 13  3 10  1]
 [17 14 24 19 21  6]
 [18 27 33 31 15  9]
 [ 0 35  5  8 26 20]
 [16 12 25  7 29 23]]
Step 7:
[[ 2 22 34 28  4 11]
 [30 32 13  3 10  1]
 [17 14 24 19 21  6]
 [18 27 33 31 15  9]
 [16 35  5  8 26 20]
 [ 0 12 25  7 29 23]]
Step 8:
[[ 2 22 34 28  4 11]
 [30 32 13

In [37]:
print(f"It took {delta} seconds to run the code")
print(f"Quality (Number of Moves): {len(solution)}")
print(f"Cost (Total States Evaluated): {cost}")

It took 125.52191209793091 seconds to run the code
Quality (Number of Moves): 578
Cost (Total States Evaluated): 1115264


## 7x7 (weighted A*)

In [38]:
PUZZLE_DIM = 7
action = namedtuple('Action', ['pos1', 'pos2'])
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in range(RANDOMIZE_STEPS):
    state = do_action(state, choice(available_actions(state)))
state

array([[44, 18, 22, 21, 36, 17, 10],
       [28, 35, 16, 38,  6, 29,  9],
       [30, 24, 32, 33, 31, 48, 34],
       [42, 27,  2, 39, 13,  0, 45],
       [ 4, 19,  5, 41, 12, 37, 11],
       [20, 43,  7, 15, 47, 25,  1],
       [46, 26,  3,  8, 40, 23, 14]])

In [39]:
# Solve and output
weight = 20
start_time = time.time()
solution, cost = solve_puzzle_weight(state, weight)
end_time = time.time()
delta = end_time - start_time
print("Solution Steps:")
x = 1
for step in solution:
    print(f"Step {x}:")
    x += 1
    print(step)

Solution Steps:
Step 1:
[[44 18 22 21 36 17 10]
 [28 35 16 38  6 29  9]
 [30 24 32 33 31  0 34]
 [42 27  2 39 13 48 45]
 [ 4 19  5 41 12 37 11]
 [20 43  7 15 47 25  1]
 [46 26  3  8 40 23 14]]
Step 2:
[[44 18 22 21 36 17 10]
 [28 35 16 38  6  0  9]
 [30 24 32 33 31 29 34]
 [42 27  2 39 13 48 45]
 [ 4 19  5 41 12 37 11]
 [20 43  7 15 47 25  1]
 [46 26  3  8 40 23 14]]
Step 3:
[[44 18 22 21 36 17 10]
 [28 35 16 38  0  6  9]
 [30 24 32 33 31 29 34]
 [42 27  2 39 13 48 45]
 [ 4 19  5 41 12 37 11]
 [20 43  7 15 47 25  1]
 [46 26  3  8 40 23 14]]
Step 4:
[[44 18 22 21  0 17 10]
 [28 35 16 38 36  6  9]
 [30 24 32 33 31 29 34]
 [42 27  2 39 13 48 45]
 [ 4 19  5 41 12 37 11]
 [20 43  7 15 47 25  1]
 [46 26  3  8 40 23 14]]
Step 5:
[[44 18 22 21 17  0 10]
 [28 35 16 38 36  6  9]
 [30 24 32 33 31 29 34]
 [42 27  2 39 13 48 45]
 [ 4 19  5 41 12 37 11]
 [20 43  7 15 47 25  1]
 [46 26  3  8 40 23 14]]
Step 6:
[[44 18 22 21 17  6 10]
 [28 35 16 38 36  0  9]
 [30 24 32 33 31 29 34]
 [42 27  2 39 13 48

In [40]:
print(f"It took {delta} seconds to run the code")
print(f"Quality (Number of Moves): {len(solution)}")
print(f"Cost (Total States Evaluated): {cost}")

It took 34.45245575904846 seconds to run the code
Quality (Number of Moves): 1226
Cost (Total States Evaluated): 220114


## 8x8 (weighted A*)

In [7]:
PUZZLE_DIM = 8
action = namedtuple('Action', ['pos1', 'pos2'])
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in range(RANDOMIZE_STEPS):
    state = do_action(state, choice(available_actions(state)))
state

array([[24, 16, 36, 11,  8, 61, 39, 15],
       [30, 35,  3, 48, 28, 33, 13, 14],
       [44, 37, 52, 53, 23,  9, 51, 19],
       [43, 60, 50, 59, 22, 34,  7,  5],
       [55, 62, 63, 41,  1, 47,  6, 40],
       [12, 27, 10,  4, 54, 46, 45,  0],
       [25, 38, 32,  2, 20, 17, 26, 49],
       [58, 31, 42, 18, 57, 29, 56, 21]])

In [8]:
# Solve and output
weight = 30
start_time = time.time()
solution, cost = solve_puzzle_weight(state, weight)
end_time = time.time()
delta = end_time - start_time
print("Solution Steps:")
x = 1
for step in solution:
    print(f"Step {x}:")
    x += 1
    print(step)

Solution Steps:
Step 1:
[[24 16 36 11  8 61 39 15]
 [30 35  3 48 28 33 13 14]
 [44 37 52 53 23  9 51 19]
 [43 60 50 59 22 34  7  5]
 [55 62 63 41  1 47  6 40]
 [12 27 10  4 54 46  0 45]
 [25 38 32  2 20 17 26 49]
 [58 31 42 18 57 29 56 21]]
Step 2:
[[24 16 36 11  8 61 39 15]
 [30 35  3 48 28 33 13 14]
 [44 37 52 53 23  9 51 19]
 [43 60 50 59 22 34  7  5]
 [55 62 63 41  1 47  6 40]
 [12 27 10  4 54 46 26 45]
 [25 38 32  2 20 17  0 49]
 [58 31 42 18 57 29 56 21]]
Step 3:
[[24 16 36 11  8 61 39 15]
 [30 35  3 48 28 33 13 14]
 [44 37 52 53 23  9 51 19]
 [43 60 50 59 22 34  7  5]
 [55 62 63 41  1 47  6 40]
 [12 27 10  4 54 46 26 45]
 [25 38 32  2 20 17 49  0]
 [58 31 42 18 57 29 56 21]]
Step 4:
[[24 16 36 11  8 61 39 15]
 [30 35  3 48 28 33 13 14]
 [44 37 52 53 23  9 51 19]
 [43 60 50 59 22 34  7  5]
 [55 62 63 41  1 47  6 40]
 [12 27 10  4 54 46 26 45]
 [25 38 32  2 20 17 49 21]
 [58 31 42 18 57 29 56  0]]
Step 5:
[[24 16 36 11  8 61 39 15]
 [30 35  3 48 28 33 13 14]
 [44 37 52 53 23  9 51

In [9]:
print(f"It took {delta} seconds to run the code")
print(f"Quality (Number of Moves): {len(solution)}")
print(f"Cost (Total States Evaluated): {cost}")

It took 33.20232176780701 seconds to run the code
Quality (Number of Moves): 1934
Cost (Total States Evaluated): 170572
